# Setting things up

In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook, tqdm
import multiprocessing as mp
import os
import time
from pathlib import Path
import numpy as np
import re

In [2]:
"""Set up the working path"""
home = os.path.expanduser("~")


# raw html paths
html_root = os.path.join(home, 'Desktop/data')
path_master_indexes = os.path.join(html_root, 'master_indexes')
path_daily_data_html = os.path.join(html_root, 'daily_data')

# filtered text paths
filtered_text_root = os.path.join(home, 'Desktop/filtered_text_data')
filtered_text_log = os.path.join(filtered_text_root, 'errors.log')
path_daily_data_text_log = os.path.join(filtered_text_root, 'daily_data')

# Split the work

## Generate the list of files
Ultimately, this will come from the S3 bucket.

In [3]:
list_html_paths = []
file_size = []
t0 = time.perf_counter()
for filename in Path(path_daily_data_html).glob('**/*.html'):
    list_html_paths.append(filename)
    file_size.append(os.path.getsize(filename))
t1 = time.perf_counter()
print("[INFO] Found {} html files in {:.3f} s ({:,.0f} files/s)"
      .format(len(list_html_paths), t1-t0, len(list_html_paths)/(t1-t0)))
print("[INFO] Total size: {:,} bytes".format(sum(file_size)))

[INFO] Found 104 html files in 0.008 s (12,900 files/s)
[INFO] Total size: 1,539,017,824 bytes


## Split the work equally among N instances

In [4]:
def balanced_split_of_paths(list_html_paths, instances):
    # In the unlikely event we have summoned more machines than html links
    nb_instances = min(len(list_html_paths), len(instances))
    instances = instances[:nb_instances]
    
    # Split the work equally
    html_indexes = np.linspace(0, len(list_html_paths), len(instances), endpoint=False)
    html_indexes = [int(i) for i in html_indexes]  # Cast to int
    #print(qtr_indexes)
    html_indexes.append(len(list_html_paths))  # Add the last element for the comprehension below
    #print(qtr_indexes)
    split_work = []
    for i in range(len(html_indexes)-1):
        split_work.append(list_html_paths[html_indexes[i]:html_indexes[i+1]])
    
    return split_work

def test_balanced_split_of_paths():
    list_html_paths = [1]*100
    list_instances = [[1]*1, [1]*8, [1]*150]  # Three key cases
    
    for instances in list_instances:
        # Test 1: test length of the list
        test_1 = balanced_split_of_paths(list_html_paths, instances)
        assert len(test_1) == min(len(list_html_paths), len(instances))

        # Test 2: verify that wp are of near equal length
        theoretical_wp_len = len(list_html_paths)/len(instances)
        for work_package in test_1:
            assert theoretical_wp_len-1 <= len(work_package) <= theoretical_wp_len+1

        # Test 3: check package length count
        counter = 0
        for work_package in test_1:
            counter += len(work_package)
        assert counter == len(list_html_paths)
    
    return True


test_balanced_split_of_paths()

True

In [5]:
list_path_html = balanced_split_of_paths(list_html_paths, [1]*1)

In [6]:
def path_html_to_path_text(path_html):
    #print(path_html)
    end_path_html = path_html.split(html_root)[1].lstrip('/')
    #print(end_path_html)
    end_path_text = end_path_html[:-5] + '.txt'
    #print(end_path_text)
    filtered_text_path = os.path.join(filtered_text_root, end_path_text)
    return filtered_text_path


In [7]:
work_package = list_path_html[0]
path_html = str(work_package[0])
path_html_to_path_text(path_html)

'/home/alex/Desktop/filtered_text_data/daily_data/20180222/100517/000119312518054235.txt'

In [8]:
def is_downloaded(filepath):
    #expected_path = master_url_to_filepath(url_idx)
    if os.path.isfile(filepath):
        return True
    else:  # Build the folder architecture if needed
        if not os.path.isdir(os.path.split(filepath)[0]):
            os.makedirs(os.path.split(filepath)[0])
        return False

In [9]:
def filter_html_file(path_html):
    # Build the file path and make sure there is not already something
    # WARNING: path_html comes as a PosixPath or WindowsPath object - cast to str first
    path_filtered_text = path_html_to_path_text(str(path_html))
    if not is_downloaded(path_filtered_text):
        # 1. Read the file
        with open(path_html, 'r', encoding="utf8", errors='ignore') as f:
            data_html = f.read()
        
        # 2. Find the PART I, II, III and IV markers
        # WARNING: This is not easy due to the amount of entangled HTML in the text.
        # Instead, I might have to parse with bs4 and then perform the search. Will take a long time.
        soup = BeautifulSoup(data_html, 'lxml')
        data_html = soup.get_text()
        
        end_toc = re.search('PART[ \n\t]*IV', data_html)
        
        if not end_toc:
            print("[INFO] Failure 1 in", path_html)  # PART IV could not be located
            return
        data_html = data_html[end_toc.end():]  # Skip the ToC references
        
        # 3. Find all mentions of "PART XX"
        pattern = re.compile('PART[ \n\t]*I[IV]{0,2}')
        #pattern = re.compile('P(<[ A-Za-z0-9\"\-\;\/\=\:]*>)*A(<[ A-Za-z0-9\"\-\;\/\=\:]*>)R(<[ A-Za-z0-9\"\-\;\/\=\:]*>)T[ \n\t]*I[IV]{0,2}')
        parts = {}
        
        # There are many variations for what separates PART from the number
        # So we normalize as many as we can
        for m in re.finditer(pattern, data_html):
            part = m.group()
            value = re.search('I[IV]{0,2}$', part)  # Search the end
            part = part[:4] + ' ' + part[value.start():]
            parts[part] = m.span()
        # parts = {m.group(): m.span() for m in re.finditer(pattern, data_html)}
        try:
            assert len(parts) == 4
            assert 'PART I' in parts
            assert 'PART II' in parts
            assert 'PART III' in parts
            assert 'PART IV' in parts
        except:
            print("[INFO] Failure 2 in {} | {}".format(path_html, parts))
            return
        
        # 4. Filter the text - get PART I, II, III but discard PART IV
        cropped_data_html = data_html[parts['PART I'][0]:parts['PART IV'][0]]
        filtered_text = cropped_data_html
        
        """[TBR] Originally, get_text() after cropping but nearly impossible due to entangled HTML
        soup = BeautifulSoup(cropped_data_html, 'lxml')
        filtered_text = soup.get_text()
        """
        
        # 5. Write to file
        with open(path_filtered_text, 'w') as f:
            f.write(filtered_text)
        return True
   
    else:  # Nothing to do if the file already exists
        return False

In [10]:
work_package = list_path_html[0]  # Let's say we are instance 0
processing_stats = {
    'count_processed': 0,
    'count_already_processed': 0
}

with mp.Pool(processes=min(mp.cpu_count(), len(work_package))) as p:
    with tqdm(total=len(work_package)) as pbar:
        for i, value in tqdm(enumerate(p.imap_unordered(filter_html_file, work_package))):
            pbar.update()
            if value:
                processing_stats['count_processed'] += 1
            else:
                processing_stats['count_already_processed'] += 1
            """
            print("[INFO] Free space: {:,} | count_processed: {} | count_already_processed: {}"
                  .format(os.statvfs(filtered_text_root).f_frsize * os.statvfs(filtered_text_root).f_bavail,
                 processing_stats['count_processed'],
                 processing_stats['count_already_processed']))
            """

  0%|          | 0/104 [00:00<?, ?it/s]
0it [00:00, ?it/s]

[INFO] Failure 2 in /home/alex/Desktop/data/daily_data/20180222/1013462/000101346218000004.html | {'PART II': (1124, 1131), 'PART III': (188802, 188810), 'PART IVI': (190077, 190085)}


  3%|▎         | 3/104 [00:11<05:46,  3.43s/it]
3it [00:11,  3.43s/it]

[INFO] Failure 2 in /home/alex/Desktop/data/daily_data/20180222/101778/000010177818000026.html | {}


  5%|▍         | 5/104 [00:13<03:34,  2.17s/it]
5it [00:13,  2.16s/it]

...

 98%|█████████▊| 102/104 [03:41<00:03,  1.51s/it]
102it [03:41,  1.51s/it]

[INFO] Failure 2 in /home/alex/Desktop/data/daily_data/20180226/1013857/000119312518058491.html | {'PART II': (16986, 16993), 'PART III': (395978, 395986), 'PART IVI': (417633, 417641), 'PART I': (1318433, 1318438)}


 99%|█████████▉| 103/104 [03:50<00:03,  3.93s/it]
103it [03:50,  3.93s/it]

[INFO] Failure 2 in /home/alex/Desktop/data/daily_data/20180315/1013272/000094627518000013.html | {}


100%|██████████| 104/104 [05:14<00:00, 27.96s/it]
104it [05:14, 27.96s/it]
